## Code Setup

In [1]:
# If True, assumes everything is running locally.
IS_LOCAL = False

# Path to main directory
REMOTE_PATH = "/content/gdrive/Shareddrives/Birds and CS/Data/CA-Final"
LOCAL_PATH = "/Users/luca/Library/CloudStorage/GoogleDrive-luca@ucsc.edu/Shared drives/Birds and CS/Data/CA-Final"
DATA_PATH = LOCAL_PATH if IS_LOCAL else REMOTE_PATH

In [2]:
import os
import sys
import warnings
warnings.filterwarnings("ignore") # Otherwise we get too much spurious output.

In [3]:
# Installs required packages
if not IS_LOCAL:
    !pip install git+https://github.com/ecoscape-earth/ecoscape-connectivity.git
    !pip install git+https://github.com/ecoscape-earth/ecoscape-utils.git

  Cloning https://github.com/ecoscape-earth/ecoscape-connectivity.git to /tmp/pip-req-build-jvs8prx2
  Running command git clone --filter=blob:none --quiet https://github.com/ecoscape-earth/ecoscape-connectivity.git /tmp/pip-req-build-jvs8prx2
  Resolved https://github.com/ecoscape-earth/ecoscape-connectivity.git to commit 23e3ef478fc073aa3da612ff8168ed9aa60d8eb6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.2 MB/s eta 0:00:00
  Created wheel for ecoscape-connectivity: filename=ecoscape_connectivity-0.0.2-py3-none-any.whl size=11054 sha256=960865e6175b68f68ad87fd7b9f16fcf4bb02af07235558dd836d6859e655550
  Stored in directory: /tmp/pip-ephem-wheel-cache-c0imryef/wheels/72/73/2c/f3b584a638e8a6a37479ac7d8823cb5a88a277b40f8c2732f4
Successfully built ecoscape-connectivity
  Cloning https://github.com

In [4]:
# Connecting to Drive.
if not IS_LOCAL:
    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [5]:
import time
from ecoscape_utilities import BirdRun
from ecoscape_connectivity import compute_connectivity
from ecoscape_connectivity.util import read_transmission_csv
from scgt import GeoTiff

## Run definitions

In [6]:
bird_run = BirdRun(DATA_PATH)

birds = []

ORIG_NUM_SIMULATIONS = 400
NUM_SIMULATIONS = 40
TRANSMISSION_EXPONENT = 1
BIRD_RUN = "Precision"

birds.append(bird_run.get_bird_run(
    "acowoo", "Acorn Woodpecker",
    run_name=BIRD_RUN,
    do_validation=False, do_gradient=True,
    hop_distance=2, num_spreads=20,
    num_simulations=NUM_SIMULATIONS,
    transmission_exponent=TRANSMISSION_EXPONENT))

birds.append(bird_run.get_bird_run(
    "acowoo", "Acorn Woodpecker",
    run_name=BIRD_RUN,
    do_validation=False, do_gradient=True,
    hop_distance=3, num_spreads=15,
    num_simulations=NUM_SIMULATIONS,
    transmission_exponent=TRANSMISSION_EXPONENT))

birds.append(bird_run.get_bird_run(
    "stejay", "Steller's Jay",
    run_name=BIRD_RUN,
    do_validation=False, do_gradient=True,
    hop_distance=1, num_spreads=6,
    num_simulations=NUM_SIMULATIONS,
    transmission_exponent=TRANSMISSION_EXPONENT))

birds.append(bird_run.get_bird_run(
    "stejay", "Steller's Jay",
    run_name=BIRD_RUN,
    do_validation=False, do_gradient=True,
    hop_distance=2, num_spreads=3,
    num_simulations=NUM_SIMULATIONS,
    transmission_exponent=TRANSMISSION_EXPONENT))


In [7]:
import numpy as np
import math

In [8]:
# Let's do the bird runs.
OUT_CONNECTIVITY = "connectivity.tif"
OUT_FLOW = "flow.tif"
NUM_RUNS = 100

t = time.time()
conn_variance = {}
flow_variance = {}
for bird in birds:

    tag = "{}_gap_{}_n_{}".format(bird.name, bird.hop_distance, bird.num_spreads)
    # Reads and translates the resistance dictionary.
    transmission_d = read_transmission_csv(bird.transmission_fn)

    # Performs a number of analysis, and measures accuracy of results.
    conn_acc_sum = None
    conn_acc_sqr = None
    flow_acc_sum = None
    flow_acc_sqr = None
    for run_idx in range(NUM_RUNS):
        # Performs the analysis
        compute_connectivity(
            habitat_fn=bird.habitat_fn,
            terrain_fn=bird.terrain_fn,
            connectivity_fn=OUT_CONNECTIVITY,
            flow_fn=OUT_FLOW,
            permeability_dict=transmission_d,
            gap_crossing=bird.hop_distance,
            num_gaps=bird.num_spreads,
            num_simulations=bird.num_simulations,
            seed_density=4,
            single_tile=False,
            tile_size=1024,
            tile_border=256,
        )
        # Reads the results back.
        with GeoTiff.from_file(OUT_CONNECTIVITY) as f:
            tile = f.get_all_as_tile()
            m = tile.m / 255 # To normalize range.
            if conn_acc_sum is None:
                conn_acc_sum = np.array(m)
                conn_acc_sqr = np.array(m) ** 2
            else:
                conn_acc_sum += np.array(m)
                conn_acc_sqr += np.array(m) ** 2
        with GeoTiff.from_file(OUT_FLOW) as f:
            tile = f.get_all_as_tile()
            m = (10 ** (tile.m / 20)) - 1
            if flow_acc_sum is None:
                flow_acc_sum = np.array(m)
                flow_acc_sqr = np.array(m) ** 2
            else:
                flow_acc_sum += np.array(m)
                flow_acc_sqr += np.array(m) ** 2
        print("Done run", run_idx, "for bird", bird.name)
    # Connectivity
    conn_bird_avg = conn_acc_sum / NUM_RUNS
    conn_bird_var = (conn_acc_sqr / NUM_RUNS) - (conn_bird_avg ** 2)
    conn_variance[tag] = conn_bird_var
    # Flow
    flow_bird_avg = flow_acc_sum / NUM_RUNS
    flow_bird_var = (flow_acc_sqr / NUM_RUNS) - (flow_bird_avg ** 2)
    flow_variance[tag] = flow_bird_var

Done run 0 for bird Acorn Woodpecker


Done run 1 for bird Acorn Woodpecker


Done run 2 for bird Acorn Woodpecker


Done run 3 for bird Acorn Woodpecker


Done run 4 for bird Acorn Woodpecker


...

Done run 94 for bird Steller's Jay


Done run 95 for bird Steller's Jay


Done run 96 for bird Steller's Jay


Done run 97 for bird Steller's Jay


Done run 98 for bird Steller's Jay
Done run 99 for bird Steller's Jay


In [9]:
std_renorm = math.sqrt(ORIG_NUM_SIMULATIONS / NUM_SIMULATIONS)

def perc_below_threshold(std, threshold, renorm=std_renorm):
    t = threshold * std_renorm
    return np.sum(std > t) / len(std)

def analyze_variance(a, values=None):
    std = np.sqrt(a.flatten())
    for t in values:
        print("Perc > {}:".format(t), 100 * perc_below_threshold(std, t))

In [10]:
for tag, v in conn_variance.items():
    print(tag, ":")
    analyze_variance(v, values=[0.5, 0.2, 0.1, 0.05, 0.02, 0.01])

Acorn Woodpecker_gap_2_n_20 :
Perc > 0.5: 0.0
Perc > 0.2: 0.0
Perc > 0.1: 0.0
Perc > 0.05: 0.0
Perc > 0.02: 0.7079399499482807
Perc > 0.01: 3.6751138535479724
Acorn Woodpecker_gap_3_n_15 :
Perc > 0.5: 0.0
Perc > 0.2: 0.0
Perc > 0.1: 0.0
Perc > 0.05: 0.0
Perc > 0.02: 0.9324139634727749
Perc > 0.01: 3.7911444771445098
Steller's Jay_gap_1_n_6 :
Perc > 0.5: 0.0
Perc > 0.2: 0.0
Perc > 0.1: 0.0
Perc > 0.05: 0.0
Perc > 0.02: 0.30014615934832267
Perc > 0.01: 3.4643178806674286
Steller's Jay_gap_2_n_3 :
Perc > 0.5: 0.0
Perc > 0.2: 0.0
Perc > 0.1: 0.0
Perc > 0.05: 0.0
Perc > 0.02: 0.5199707401036148
Perc > 0.01: 3.3573058554025406


In [11]:
for tag, v in flow_variance.items():
    print(tag, ":")
    analyze_variance(v, values=[1, 2, 5, 10, 20, 30, 40])

Acorn Woodpecker_gap_2_n_20 :
Perc > 1: 6.670829969292724
Perc > 2: 6.295632255792673
Perc > 5: 5.625668512359684
Perc > 10: 4.8337635101305585
Perc > 20: 3.4177685004883656
Perc > 30: 1.8342537775515175
Perc > 40: 0.4388534028342622
Acorn Woodpecker_gap_3_n_15 :
Perc > 1: 6.768112718665977
Perc > 2: 6.386393787837324
Perc > 5: 5.689339216008542
Perc > 10: 4.860844328983252
Perc > 20: 3.3990356405797284
Perc > 30: 1.8146600969364193
Perc > 40: 0.49552243108579713
Steller's Jay_gap_1_n_6 :
Perc > 1: 9.028362640941873
Perc > 2: 7.924234965616218
Perc > 5: 0.4556744400111066
Perc > 10: 5.004771549195002e-06
Perc > 20: 0.0
Perc > 30: 0.0
Perc > 40: 0.0
Steller's Jay_gap_2_n_3 :
Perc > 1: 8.704739098256278
Perc > 2: 7.322691454032275
Perc > 5: 0.0007256918746332753
Perc > 10: 0.0
Perc > 20: 0.0
Perc > 30: 0.0
Perc > 40: 0.0
